#### One-hot encoding to origin column

In [2]:
import pandas as pd

rasff_flag=pd.read_csv(r"F:\Final_project\rasff_new3.csv")

In [4]:
rasff_flag.head()

,reference,category,type,subject,date,notifying_country,classification,risk_decision,distribution,forAttention,forFollowUp,operator,origin,hazards,year,month,Hazard_Type,Regulatory_Issue_Flag,category_Alerts_Last_180d,origin_Alerts_Last_365d
0,2020.0001,fruits and vegetables,food,Aflatoxins in dried figs from Turkey,2020-01-02 09:24:48,France,border rejection notification,serious,NaN,NaN,NaN,Türkiye,Türkiye,Aflatoxins B1,2020,1,Mycotoxins,0,0,0
1,2020.0004,fruits and vegetables,food,aflatoxins in dried figs from Turkey,2020-01-02 11:06:54,Italy,border rejection notification,serious,NaN,NaN,NaN,"Italy,Türkiye",Türkiye,aflatoxin total,2020,1,Mycotoxins,0,1,1
2,2020.0007,fruits and vegetables,food,fosthiazate in fresh peppers from Turkey,2020-01-02 12:35:16,Bulgaria,border rejection notification,serious,NaN,NaN,NaN,"Germany,Türkiye",Türkiye,fosthiazate,2020,1,Chemical/Contaminants,0,2,2
3,2020.0008,meat and meat products (other than poultry),food,Salmonella spp. (in 1 out of 5 samples) in Hot...,2020-01-02 12:35:36,Poland,alert notification,serious,Estonia,Poland,NaN,"Estonia,Poland",Poland,Salmonella enteritidis,2020,1,Microbiological,0,0,0
4,2020.0010,food contact materials,food contact material,migration of formaldehyde form bamboo mug from...,2020-01-02 13:42:42,Poland,alert notification,serious,"Belarus,Lithuania,Poland,Slovakia",NaN,"Lithuania,Slovakia","China,Poland",China,formaldehyde migration,2020,1,Migration from Packaging,0,0,0


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

def one_hot_encode_top_n_origins_mlb(df: pd.DataFrame, column_name: str, top_n: int = 50) -> pd.DataFrame:
    """
    Performs One-Hot Encoding (OHE) for the top N most frequent unique countries 
    using MultiLabelBinarizer, while ignoring all other rare countries.
    """
    
    print(f"Starting MLB OHE for top {top_n} unique origins from column '{column_name}'...")
    
    # 1. Prepare and explode data to count all unique countries
    df[column_name] = df[column_name].fillna('')
    
    # Create the list column (your original first step)
    df['origin_list'] = df[column_name].apply(lambda x: [c.strip() for c in str(x).split(',') if c.strip()])

    # Explode to count frequencies
    all_countries = df['origin_list'].explode()
    
    # 2. Identify the Top N Most Frequent Countries
    top_countries = all_countries.value_counts().nlargest(top_n).index.tolist()
    
    print(f"Top {top_n} countries identified (Total unique: {all_countries.nunique()}): {top_countries[:5]}...")
    
    # 3. Filter the 'origin_list' to include ONLY the top N countries
    def filter_origins(country_list):
        return [country for country in country_list if country in top_countries]

    df['filtered_origin_list'] = df['origin_list'].apply(filter_origins)

    # 4. Apply MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    
    # Fit and transform using the filtered list
    origin_dummies = pd.DataFrame(
        mlb.fit_transform(df['filtered_origin_list']),
        columns=[f"Origin_OHE_{c}" for c in mlb.classes_],
        index=df.index
    )

    # 5. Concatenate and clean up
    df = pd.concat([df, origin_dummies], axis=1)
    
    # Drop temporary columns
    df = df.drop(columns=['origin_list', 'filtered_origin_list'])
    
    print(f"Successfully added {len(mlb.classes_)} Top N OHE features.")
    
    return df

# --- Demonstration Usage ---

# Create sample data that mimics your multi-value 'origin' column structure
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'origin': [
        'China, USA',      # 2 countries
        'Brazil', 
        'India, China, Rare_A', # 3 countries, 1 rare
        'USA', 
        'China', 
        'Brazil, India', 
        'USA, Brazil',
        'India',
        'China',
        'Rare_B', # Rare
        'China',
        'USA',
        'Brazil',
        'India',
        np.nan 
    ],
    'risk_score': [10, 5, 8, 12, 11, 4, 9, 7, 6, 1, 2, 3, 4, 5, 6]
}

sample_df = pd.DataFrame(data)

# Run the function to create OHE features for the top 3 most frequent origins
df_engineered = one_hot_encode_top_n_origins_mlb(
    df=sample_df.copy(), # Work on a copy
    column_name='origin', 
    top_n=3
)

print("\n--- Engineered DataFrame (Top 3 Origins OHE) ---")
# Only the columns for China, USA, Brazil, and India should appear. Rare_A/B are ignored.
print(df_engineered[['id', 'origin'] + [col for col in df_engineered.columns if col.startswith('Origin_OHE')]].to_markdown(index=False))


Starting MLB OHE for top 3 unique origins from column 'origin'...
Top 3 countries identified (Total unique: 6): ['China', 'USA', 'Brazil']...
Successfully added 3 Top N OHE features.

--- Engineered DataFrame (Top 3 Origins OHE) ---
|   id | origin               |   Origin_OHE_Brazil |   Origin_OHE_China |   Origin_OHE_USA |
|-----:|:---------------------|--------------------:|-------------------:|-----------------:|
|    1 | China, USA           |                   0 |                  1 |                1 |
|    2 | Brazil               |                   1 |                  0 |                0 |
|    3 | India, China, Rare_A |                   0 |                  1 |                0 |
|    4 | USA                  |                   0 |                  0 |                1 |
|    5 | China                |                   0 |                  1 |                0 |
|    6 | Brazil, India        |                   1 |                  0 |                0 |
|    7 | USA, B

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

def one_hot_encode_top_n_origins_mlb(df: pd.DataFrame, column_name: str, top_n: int = 50) -> pd.DataFrame:
    """
    Performs One-Hot Encoding (OHE) for the top N most frequent unique countries 
    using MultiLabelBinarizer, while ignoring all other rare countries.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing the multi-value column.
        column_name (str): The name of the column (e.g., 'origin') to process.
        top_n (int): The number of top unique countries to encode.

    Returns:
        pd.DataFrame: The original DataFrame with new binary columns added 
                      (e.g., 'Origin_OHE_USA', 'Origin_OHE_China', etc.).
    """
    
    print(f"Starting MLB OHE for top {top_n} unique origins from column '{column_name}'...")
    
    # 1. Prepare and explode data to count all unique countries
    # Replace NaN/empty with empty string and create the list of stripped countries
    df[column_name] = df[column_name].fillna('')
    df['origin_list'] = df[column_name].apply(
        lambda x: [c.strip() for c in str(x).split(',') if c.strip()]
    )

    # Explode to count frequencies
    all_countries = df['origin_list'].explode()
    
    # 2. Identify the Top N Most Frequent Countries
    # Handle the case where there are fewer unique countries than top_n requested
    top_countries = all_countries.value_counts().nlargest(top_n).index.tolist()
    
    print(f"Top {len(top_countries)} countries identified (Total unique: {all_countries.nunique()}): {top_countries[:5]}...")
    
    # 3. Filter the 'origin_list' to include ONLY the top N countries
    def filter_origins(country_list, top_set):
        # Use set lookup for very fast filtering
        return [country for country in country_list if country in top_set]

    # Convert the list to a set for O(1) average-time complexity lookups
    top_countries_set = set(top_countries)
    
    df['filtered_origin_list'] = df['origin_list'].apply(
        lambda x: filter_origins(x, top_countries_set)
    )

    # 4. Apply MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    
    # Fit and transform using the filtered list
    origin_dummies = pd.DataFrame(
        mlb.fit_transform(df['filtered_origin_list']),
        columns=[f"Origin_OHE_{c}" for c in mlb.classes_],
        index=df.index
    )

    # 5. Concatenate and clean up
    df = pd.concat([df, origin_dummies], axis=1)
    
    # Drop temporary columns
    df = df.drop(columns=['origin_list', 'filtered_origin_list'])
    
    # Ensure OHE columns are integer type (0 or 1)
    new_ohe_cols = [col for col in df.columns if col.startswith('Origin_OHE_')]
    df[new_ohe_cols] = df[new_ohe_cols].astype(int)

    print(f"Successfully added {len(mlb.classes_)} Top N OHE features.")
    
    return df



In [5]:
TOP_N_COUNTRIES = 50 

rasff_flag_with_ohe = one_hot_encode_top_n_origins_mlb(
    df=rasff_flag.copy(), 
    column_name='origin', 
    top_n=TOP_N_COUNTRIES
)

Starting MLB OHE for top 50 unique origins from column 'origin'...
Top 50 countries identified (Total unique: 166): ['Türkiye', 'India', 'Poland', 'China', 'Netherlands']...
Successfully added 50 Top N OHE features.


In [6]:
rasff_flag_with_ohe.head()

,reference,category,type,subject,date,notifying_country,classification,risk_decision,distribution,forAttention,...,Origin_OHE_Switzerland,Origin_OHE_Syria,Origin_OHE_Thailand,Origin_OHE_Türkiye,Origin_OHE_Uganda,Origin_OHE_Ukraine,Origin_OHE_United Kingdom,Origin_OHE_United States,Origin_OHE_Vietnam,Origin_OHE_unknown origin
0,2020.0001,fruits and vegetables,food,Aflatoxins in dried figs from Turkey,2020-01-02 09:24:48,France,border rejection notification,serious,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
1,2020.0004,fruits and vegetables,food,aflatoxins in dried figs from Turkey,2020-01-02 11:06:54,Italy,border rejection notification,serious,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
2,2020.0007,fruits and vegetables,food,fosthiazate in fresh peppers from Turkey,2020-01-02 12:35:16,Bulgaria,border rejection notification,serious,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
3,2020.0008,meat and meat products (other than poultry),food,Salmonella spp. (in 1 out of 5 samples) in Hot...,2020-01-02 12:35:36,Poland,alert notification,serious,Estonia,Poland,...,0,0,0,0,0,0,0,0,0,0
4,2020.0010,food contact materials,food contact material,migration of formaldehyde form bamboo mug from...,2020-01-02 13:42:42,Poland,alert notification,serious,"Belarus,Lithuania,Poland,Slovakia",NaN,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Replace the original DataFrame with the new one containing OHE features
rasff_flag = rasff_flag_with_ohe
del rasff_flag_with_ohe # Clean up memory

In [8]:
rasff_flag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26823 entries, 0 to 26822
Data columns (total 70 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   reference                  26823 non-null  float64
 1   category                   26823 non-null  object 
 2   type                       26823 non-null  object 
 3   subject                    26823 non-null  object 
 4   date                       26823 non-null  object 
 5   notifying_country          26823 non-null  object 
 6   classification             26823 non-null  object 
 7   risk_decision              26823 non-null  object 
 8   distribution               18369 non-null  object 
 9   forAttention               14687 non-null  object 
 10  forFollowUp                13510 non-null  object 
 11  operator                   26773 non-null  object 
 12  origin                     26823 non-null  object 
 13  hazards                    19919 non-null  obj

In [9]:
# --- Code to be placed immediately after: rasff_flag = rasff_flag_with_ohe ---

# 1. Dynamically identify all the newly created OHE columns
new_ohe_cols = [col for col in rasff_flag.columns if col.startswith('Origin_OHE_')]

# 2. Sum the binary OHE columns across the row (axis=1)
# The resulting column counts how many top-50 origins are involved in a single incident.
rasff_flag['Aggregated_Origin_Risk_Flag'] = rasff_flag[new_ohe_cols].sum(axis=1)

print(f"Created new aggregated feature: Aggregated_Origin_Risk_Flag.")
print(f"New feature added. Maximum number of top-50 origins in a single incident: {rasff_flag['Aggregated_Origin_Risk_Flag'].max()}")

# 3. Optional: Decide if you still need the individual OHE columns
# While aggregation provides a single feature, the individual OHE columns 
# (e.g., Origin_OHE_China) are still crucial for the XGBoost model to learn 
# country-specific risks. It's recommended to keep BOTH.

# If you only wanted the aggregated feature, you would drop the OHE columns here:
# rasff_flag.drop(columns=new_ohe_cols, inplace=True)

Created new aggregated feature: Aggregated_Origin_Risk_Flag.
New feature added. Maximum number of top-50 origins in a single incident: 13


In [10]:
rasff_flag.head()

,reference,category,type,subject,date,notifying_country,classification,risk_decision,distribution,forAttention,...,Origin_OHE_Syria,Origin_OHE_Thailand,Origin_OHE_Türkiye,Origin_OHE_Uganda,Origin_OHE_Ukraine,Origin_OHE_United Kingdom,Origin_OHE_United States,Origin_OHE_Vietnam,Origin_OHE_unknown origin,Aggregated_Origin_Risk_Flag
0,2020.0001,fruits and vegetables,food,Aflatoxins in dried figs from Turkey,2020-01-02 09:24:48,France,border rejection notification,serious,NaN,NaN,...,0,0,1,0,0,0,0,0,0,1
1,2020.0004,fruits and vegetables,food,aflatoxins in dried figs from Turkey,2020-01-02 11:06:54,Italy,border rejection notification,serious,NaN,NaN,...,0,0,1,0,0,0,0,0,0,1
2,2020.0007,fruits and vegetables,food,fosthiazate in fresh peppers from Turkey,2020-01-02 12:35:16,Bulgaria,border rejection notification,serious,NaN,NaN,...,0,0,1,0,0,0,0,0,0,1
3,2020.0008,meat and meat products (other than poultry),food,Salmonella spp. (in 1 out of 5 samples) in Hot...,2020-01-02 12:35:36,Poland,alert notification,serious,Estonia,Poland,...,0,0,0,0,0,0,0,0,0,1
4,2020.0010,food contact materials,food contact material,migration of formaldehyde form bamboo mug from...,2020-01-02 13:42:42,Poland,alert notification,serious,"Belarus,Lithuania,Poland,Slovakia",NaN,...,0,0,0,0,0,0,0,0,0,1


In [16]:
rasff_flag[rasff_flag['Aggregated_Origin_Risk_Flag']>1].count()

reference                      962
category                       962
type                           962
subject                        962
date                           962
                              ... 
Origin_OHE_United Kingdom      962
Origin_OHE_United States       962
Origin_OHE_Vietnam             962
Origin_OHE_unknown origin      962
Aggregated_Origin_Risk_Flag    962
Length: 71, dtype: int64

In [17]:
rasff_flag.to_csv("rasff_new4_origin.csv", index=False)

In [18]:
rasff=pd.read_csv(r"F:\Final_project\rasff_new4_origin.csv")

In [19]:
rasff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26823 entries, 0 to 26822
Data columns (total 71 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   reference                    26823 non-null  float64
 1   category                     26823 non-null  object 
 2   type                         26823 non-null  object 
 3   subject                      26823 non-null  object 
 4   date                         26823 non-null  object 
 5   notifying_country            26823 non-null  object 
 6   classification               26823 non-null  object 
 7   risk_decision                26823 non-null  object 
 8   distribution                 18369 non-null  object 
 9   forAttention                 14687 non-null  object 
 10  forFollowUp                  13510 non-null  object 
 11  operator                     26773 non-null  object 
 12  origin                       26823 non-null  object 
 13  hazards         

In [20]:
rasff['Aggregated_Origin_Risk_Flag'].value_counts()

Aggregated_Origin_Risk_Flag
1     23340
0      2521
2       825
3        99
4        25
5         9
9         1
7         1
13        1
6         1
Name: count, dtype: int64